In [353]:
import pandas as pd
import numpy as np

In [354]:
dataset = pd.read_csv('cyberbullying_tweets.csv')
dataset.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [355]:
dataset.tail()

,tweet_text,cyberbullying_type
47687,"Black ppl aren't expected to do anything, depe...",ethnicity
47688,Turner did not withhold his disappointment. Tu...,ethnicity
47689,I swear to God. This dumb nigger bitch. I have...,ethnicity
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,ethnicity
47691,Bro. U gotta chill RT @CHILLShrammy: Dog FUCK ...,ethnicity


In [356]:
dataset.shape

(47692, 2)

In [357]:
dataset.isnull().sum()

tweet_text            0
cyberbullying_type    0
dtype: int64

In [358]:
dataset2 = dataset.copy()

In [359]:
translator = str.maketrans('', '', string.punctuation)
stop_words = set(stopwords.words('english'))

In [360]:
dataset2.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [361]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy

In [362]:
translator = str.maketrans('','', string.punctuation)
stop_words = set(stopwords.words('english'))
nlp = spacy.load('en_core_web_sm')

In [363]:
for i in range(dataset.shape[0]):
    words = word_tokenize(dataset2.loc[i,'tweet_text'])
    lw_wd = [word.lower() for word in words ]
    no_pun_wd = [word.translate(translator) for word in lw_wd]
    filtered_wd = [word for word in no_pun_wd if word not in stop_words]
    doc = nlp(" ".join(filtered_wd))
    lemmatized_wd = [word.lemma_ for word in doc]
    dataset2.loc[i,'tweet_text'] = lemmatized_wd
    
    

In [364]:
dataset2.head()

,tweet_text,cyberbullying_type
0,"[word, , katandandre, , food, crapilicious, ...",not_cyberbullying
1,"[ , aussietv, white, , mkr, , theblock, , ...",not_cyberbullying
2,"[ , xochitlsuckkks, classy, whore, , red, vel...",not_cyberbullying
3,"[ , jasongio, meh, , p, thank, head, , conc...",not_cyberbullying
4,"[ , rudhoeenglish, isis, account, pretend, kur...",not_cyberbullying


In [365]:
for i in range(dataset2.shape[0]):
    words = dataset2.loc[i,'tweet_text']
    s  = ""
    for word in words:
        s= s+" "+str(word)
    dataset2.loc[i,'tweet_text'] = s

In [366]:
dataset2.head()

,tweet_text,cyberbullying_type
0,word katandandre food crapilicious mkr,not_cyberbullying
1,aussietv white mkr theblock imaceleb...,not_cyberbullying
2,xochitlsuckkks classy whore red velvet cu...,not_cyberbullying
3,jasongio meh p thank head concern anot...,not_cyberbullying
4,rudhoeenglish isis account pretend kurdish ...,not_cyberbullying


In [367]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [368]:
X_train,X_test,y_train,y_test = train_test_split(dataset2.tweet_text,dataset.cyberbullying_type,test_size=0.2,random_state=23)

In [369]:
tdf = TfidfVectorizer()
X_train_tdf = tdf.fit_transform(X_train)
X_test_tdf = tdf.transform(X_test)

In [370]:
X_train_tdf

<38153x49398 sparse matrix of type '<class 'numpy.float64'>'
	with 490285 stored elements in Compressed Sparse Row format>

In [371]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [372]:
mnb = MultinomialNB()
mnb.fit(X_train_tdf,y_train)

MultinomialNB()

In [373]:
logistic = LogisticRegression()
logistic.fit(X_train_tdf,y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [374]:
dtree = DecisionTreeClassifier()
dtree.fit(X_train_tdf,y_train)

DecisionTreeClassifier()

In [375]:
rtree = RandomForestClassifier()
rtree.fit(X_train_tdf,y_train)

RandomForestClassifier()

In [376]:
y_pred_mnb = mnb.predict(X_test_tdf)
y_pred_log = logistic.predict(X_test_tdf)
y_pred_dtree = dtree.predict(X_test_tdf)
y_pred_rtree = rtree.predict(X_test_tdf)

In [377]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [378]:
print("Multinominal Naive Bayes:",accuracy_score(y_test,y_pred_mnb))
print("Logistic:",accuracy_score(y_test,y_pred_log))
print("Decision Tree Classifier:",accuracy_score(y_test,y_pred_dtree))
print("Random forest Classifier:",accuracy_score(y_test,y_pred_rtree))

Multinominal Naive Bayes: 0.7543767690533599
Logistic: 0.8268162281161547
Decision Tree Classifier: 0.7903344166055142
Random forest Classifier: 0.8209455917811092


In [379]:
confusion_matrix(y_test,y_pred_mnb)

array([[1596,    8,    4,    2,    2,   11],
       [  63, 1373,    9,    0,    7,   50],
       [  47,   54, 1379,   78,   41,   43],
       [ 317,   93,  114,  568,  353,  179],
       [ 294,  134,  137,  172,  727,   96],
       [  18,    8,    4,    4,    1, 1553]])

In [380]:
confusion_matrix(y_test,y_pred_log)

array([[1579,    5,    5,   16,   16,    2],
       [   0, 1470,    3,    7,   20,    2],
       [   4,    6, 1380,  125,  117,   10],
       [  59,   12,   56,  898,  531,   68],
       [  42,   20,   59,  375, 1047,   17],
       [   0,    2,    3,   51,   19, 1513]])

In [381]:
confusion_matrix(y_test,y_pred_dtree)

array([[1568,    1,    1,   28,   21,    4],
       [   2, 1460,    7,    9,   21,    3],
       [   3,    3, 1383,  103,  142,    8],
       [  17,    9,  106,  771,  659,   62],
       [  11,    7,  100,  563,  873,    6],
       [   2,    7,    8,   63,   24, 1484]])

In [382]:
confusion_matrix(y_test,y_pred_rtree)

array([[1580,    1,    2,   18,   20,    2],
       [   1, 1473,    2,    4,   19,    3],
       [   2,    1, 1363,  116,  152,    8],
       [  23,    2,   53,  812,  666,   68],
       [  15,    6,   70,  390, 1071,    8],
       [   0,    1,    6,   25,   24, 1532]])

# Report:
The best model to fit from accuracy is looks to be logistic and random forest